In [ ]:
import sys
sys.executable

'c:\\Users\\carlo\\miniforge3\\envs\\dtu02452\\python.exe'

In [23]:
from pathlib import Path
import pandas as pd
import mne
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
import pyedflib
from pathlib import Path


ROOT = Path.cwd().parent
DATA_DIR = ROOT / "Data" / "cap-sleep-database-1.0.0"

print(DATA_DIR)
print("Exists:", DATA_DIR.exists())

rbd_files = sorted(DATA_DIR.glob("rbd*.edf"))
print("RBD EDFs:", len(rbd_files))
rbd_files[:10]

edf_file = rbd_files[0]
print("Using:", edf_file.name)

raw = mne.io.read_raw_edf(edf_file, preload=False, verbose=False)
raw


 


c:\Users\carlo\OneDrive - Universidade de Lisboa\Documents\GitHub\heart-lung-brain-coupling-for-RBD\Data\cap-sleep-database-1.0.0
Exists: True
RBD EDFs: 22
Using: rbd1.edf


C:\Users\carlo\AppData\Local\Temp\ipykernel_38284\3201281601.py:24: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=False, verbose=False)
C:\Users\carlo\AppData\Local\Temp\ipykernel_38284\3201281601.py:24: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(edf_file, preload=False, verbose=False)
C:\Users\carlo\AppData\Local\Temp\ipykernel_38284\3201281601.py:24: RuntimeWarning: Highpass cutoff frequency 10.0 is greater than lowpass cutoff frequency 3.0, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(edf_file, preload=False, verbose=False)


<RawEDF | rbd1.edf, 15 x 15498752 (30271.0 s), ~16 KiB, data not loaded>

In [18]:

edf_file = (
    Path("C:/Users/carlo/OneDrive - Universidade de Lisboa/Documents/GitHub")
    / "heart-lung-brain-coupling-for-RBD"
    / "Data"
    / "cap-sleep-database-1.0.0"
    / "rbd20.edf"
)

raw = mne.io.read_raw_edf(edf_files, preload=False, verbose=False)
raw

FileNotFoundError: File does not exist: "c:\Users\carlo\OneDrive - Universidade de Lisboa\Documents\GitHub\heart-lung-brain-coupling-for-RBD\Notebooks\Data\cap-sleep-database-1.0.0\ins9.edf"